[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Moorea-AI/AI4ENG_UDEA/blob/main/01_Exploración_de_datos.ipynb)

EXPLORACIÓN DE DATOS PARA EL DATASET "Fraude en Transacciones Electrónicas" TOMADO DE: https://www.kaggle.com/competitions/ieee-fraud-detection/data?select=test_transaction

Presentado por: </br>
Aura Luz Moreno Díaz, Estudiante de Ingeniería Industrial </br>
Evelyn Zharick Saez Gallego, Estudiante de Ingeniería Ambiental</br>
Para la materia:<br>
INTRODUCCIÓN A LA INTELIGENCIA ARTIFICIAL PARA LAS CIENCIAS Y LA INGENIERIA<BR>
Con el profesor: RAÚL RAMOS POLLAN

In [ ]:
#PASO 1: Importación de librerias que serán necesarias para la exploración de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import requests
import json

#Importacion de librerias para manipulación de datos para Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit

from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,KFold,GroupKFold
import lightgbm as lgb
import gc

import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Cargando credenciales de Kaggel
{"username":"mooreamed","key":"e97f8db473e00551f7aea527af024391"}
with open('kaggle.json', 'w') as file:
    json.dump(data, file, indent=4)

!cp kaggle.json ~/.kaggle/

In [ ]:
# Se descarga el dataset
!kaggle datasets download -d niangmohamed/ieeecis-fraud-detection

# Se descomprime el dataset
!unzip ieeecis-fraud-detection.zip

 96% 113M/118M [00:01<00:00, 83.6MB/s] 
100% 118M/118M [00:01<00:00, 75.6MB/s]
Archive:  ieeecis-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


In [ ]:
#Verificamos que se este funcionando la conexión de la API
!kaggle datasets list

ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
salvatorerastelli/spotify-and-youtube                          Spotify and Youtube                                   9MB  2023-03-20 15:43:25           9136        334  1.0              
arnabchaki/data-science-salaries-2023                          Data Science Salaries 2023  💸                        25KB  2023-04-13 09:55:16           2507         69  1.0              
erdemtaha/cancer-data                                          Cancer Data                                          49KB  2023-03-22 07:57:00           4237         97  1.0              
evangower/premier-league-2022-2023                             Pr

# ANALISIS DE LOS DATOS

###**TABLA IDENTITY** <BR>
Las variables en esta tabla son información de identidad: 
<BR><BR>
información de conexión de red (IP, ISP, Proxy, etc.) y firma digital (UA/
navegador/OS/versión, etc.) asociada con las transacciones.<BR><BR>
Son recopilados por el sistema de protección contra fraudes de Vesta y los socios de seguridad digital.<BR><BR>
(Los nombres de los campos están enmascarados y no se proporcionará el diccionario por pares para la protección de la privacidad y el acuerdo del contrato)<BR><BR>

TransactionID<br>
id_12 - id_38<BR>
DeviceType<BR>
DeviceInfo<BR>

<BR><BR>


###**TABLA TRANSACCIONES:** <BR>

TransactionDT: timedelta de una fecha y hora de referencia determinada (no una marca de tiempo real). timedelta de una fecha y hora de referencia dada (no una marca de tiempo real). El primer valor de TransactionDT es 86400, que corresponde a la cantidad de segundos en un día (60 * 60 * 24 = 86400), así que creo que la unidad es segundos. Usando esto, sabemos que los datos abarcan 6 meses, ya que el valor máximo es 15811131, que correspondería al día 183”
<BR>

TransactionAMT: monto del pago de la transacción en USD<BR>

ProductCD: código de producto, el producto para cada transacción<BR>

card1 - card6: información de la tarjeta de pago, como tipo de tarjeta, categoría de tarjeta, banco emisor, país, etc.<BR>

dirección: dirección addr1 como región de facturación, addr2 como país de facturación<BR>

distancia: distancias entre (no limitadas) la dirección de facturación, la dirección postal, el código postal, la dirección IP, el área telefónica, etc
<BR>

P_ y (R__) emaildomain: dominio de correo electrónico del comprador y del destinatario<BR>

C1-C14: conteo, como cuántas direcciones se encuentran asociadas con la tarjeta de pago, etc. El significado real está enmascarado.<BR>

D1-D15: timedelta, como días entre transacciones anteriores, etc.<BR>

M1-M9: coincidencia, como nombres en la tarjeta y dirección, etc.<BR>

Vxxx: características completas diseñadas por Vesta, que incluyen clasificación, conteo y otras relaciones de entidad.<BR><BR>

Características categóricas:<BR>

ProductCD<BR>
card1 - card6<BR>
addr1, addr2<BR>
P_emaildomain<BR>
R_emaildomain<BR>
M1 - M9<BR>




In [21]:
#Vamos a leer los datos que tiene el dataset para empezar con el análisis explorartorio
df = pd.read_csv('test_identity.csv')
df.head(10)

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,ie 11.0 for tablet,NaN,NaN,NaN,F,T,T,F,desktop,Trident/7.0
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G9650 Build/R16NW
5,3663622,-5.0,36786.0,NaN,NaN,4.0,-2.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,2208x1242,match_status:2,T,F,F,T,mobile,iOS Device
6,3663624,-5.0,12577.0,0.0,0.0,2.0,-2.0,NaN,NaN,0.0,...,mobile safari 11.0,32.0,2208x1242,match_status:2,T,F,F,T,mobile,iOS Device
7,3663626,-5.0,44726.0,0.0,0.0,0.0,-2.0,NaN,NaN,0.0,...,mobile safari 11.0,32.0,2208x1242,match_status:2,T,F,F,T,mobile,iOS Device
8,3663629,-5.0,125997.0,0.0,0.0,2.0,-2.0,NaN,NaN,0.0,...,mobile safari 11.0,32.0,2208x1242,match_status:2,T,F,F,T,mobile,iOS Device
9,3663658,-5.0,98741.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,2048x1536,match_status:2,T,F,F,F,mobile,iOS Device


COLUMNAS DE LA TABLA IDENTIDAD:
TransactionID<br>
id_12 - id_38<BR>
DeviceType<BR>
DeviceInfo<BR>

In [22]:
#Se imprimen las columnas para poder identificarlas y seleccionar cuáles serán susceptibles de análisis. Para esto, tomaremos solamente la tabla de identidad

df.columns


Index(['TransactionID', 'id-01', 'id-02', 'id-03', 'id-04', 'id-05', 'id-06',
       'id-07', 'id-08', 'id-09', 'id-10', 'id-11', 'id-12', 'id-13', 'id-14',
       'id-15', 'id-16', 'id-17', 'id-18', 'id-19', 'id-20', 'id-21', 'id-22',
       'id-23', 'id-24', 'id-25', 'id-26', 'id-27', 'id-28', 'id-29', 'id-30',
       'id-31', 'id-32', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')

In [23]:
#Para cumplir el requerimiento solicitado por el trabajo académico, tenemos 41 columnas y 141.907 instancias
df.shape

(141907, 41)

In [24]:
#Y la info exacta del DF
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141907 entries, 0 to 141906
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  141907 non-null  int64  
 1   id-01          141907 non-null  float64
 2   id-02          136976 non-null  float64
 3   id-03          66481 non-null   float64
 4   id-04          66481 non-null   float64
 5   id-05          134750 non-null  float64
 6   id-06          134750 non-null  float64
 7   id-07          5059 non-null    float64
 8   id-08          5059 non-null    float64
 9   id-09          74338 non-null   float64
 10  id-10          74338 non-null   float64
 11  id-11          136778 non-null  float64
 12  id-12          141907 non-null  object 
 13  id-13          130286 non-null  float64
 14  id-14          71357 non-null   float64
 15  id-15          136977 non-null  object 
 16  id-16          125747 non-null  object 
 17  id-17          135966 non-nul

In [26]:
#Debemos verificar cuántos valores nulos tenemos en la tabla, para esto hacemos el chequeo asi:

Nulos_NoNulos= df.isna().any().any()#Vamos a encontrar cuantos valores NaN existen en nuestro dataframe
Total_Nulos = df.isna().sum().sum()#Vamos a sumar el total de valores NaN presentes en el dataframe

print("Hay valores nulos?: "+str(Nulos_NoNulos))
print("Total de nulos: "+str(Total_Nulos))

Hay valores nulos?: True
Total de nulos: 2105385


Según el dato anterior, tenemos 2.105.385 de datos nulos, es decir, superior al 5%, ya que tenemos un total de 5.818.187 datos. <br>
Con esto se cumple la condición solicitada en el trabajo académico

In [27]:
df.isnull().sum() #Columnas con valores nulos

TransactionID         0
id-01                 0
id-02              4931
id-03             75426
id-04             75426
id-05              7157
id-06              7157
id-07            136848
id-08            136848
id-09             67569
id-10             67569
id-11              5129
id-12                 0
id-13             11621
id-14             70550
id-15              4930
id-16             16160
id-17              5941
id-18             91032
id-19              6001
id-20              6274
id-21            136848
id-22            136845
id-23            136845
id-24            137167
id-25            136868
id-26            136860
id-27            136845
id-28              5129
id-29              5129
id-30             71248
id-31              5282
id-32             71236
id-33             71236
id-34             69732
id-35              4930
id-36              4930
id-37              4930
id-38              4930
DeviceType         4976
DeviceInfo        26850
dtype: int64

In [28]:
df.isnull().mean() * 100 #Promedio en las columnas con valores nulos

TransactionID     0.000000
id-01             0.000000
id-02             3.474811
id-03            53.151712
id-04            53.151712
id-05             5.043444
id-06             5.043444
id-07            96.434989
id-08            96.434989
id-09            47.614987
id-10            47.614987
id-11             3.614339
id-12             0.000000
id-13             8.189166
id-14            49.715659
id-15             3.474106
id-16            11.387740
id-17             4.186545
id-18            64.149055
id-19             4.228826
id-20             4.421205
id-21            96.434989
id-22            96.432875
id-23            96.432875
id-24            96.659784
id-25            96.449083
id-26            96.443445
id-27            96.432875
id-28             3.614339
id-29             3.614339
id-30            50.207530
id-31             3.722156
id-32            50.199074
id-33            50.199074
id-34            49.139225
id-35             3.474106
id-36             3.474106
i